# Script de pré-processamento para as bases de dados do ENEM

**Autor**: Rafael Victor Araujo Bernardes - rafaelvictor.bernardes@gmail.com

## Introdução

O seguinte _script_ tem como propósito realizar a preparação dos dados para a subsequente aplicação dos métodos de clusterização e seleção de atributos propostos em meu Trabalho de Conclusão de Curso (TCC). Para tanto, o _script_ realizará sequencialmente o processamento e as transformações necessárias dos dados. O processamento dos dados é uma etapa que tem como objetivo geral: limpar, corrigir ou remover dados inconsistentes, tratar dados ausentes e identificar anomalias (_outliers_). Já o processo de transformação dos dados visa realizar a normalização, agregação, criação de novos atributos, redução e sintetização dos dados, entre outros. Estas etapas correspondem, respectivamente, à segunda e à terceira etapa do método _Knowledge Discovery in Databases_ (KDD) de Fayyad et al. (1996).

A princípio, o código aqui desenvolvido será aplicado nas bases de dados do ENEM referentes aos anos de 2022, 2020 e 2019. A escolha desses anos deve-se à alta correção dos atributos presentes nessas bases. Além disso, cada um desses anos está correlacionado a algum evento histórico cujo impacto na distribuição dos dados deseja-se investigar.

Espera-se que ao final da execução deste script, todas as bases alvo sejam transformada em bases menores, corrigidas, normalizadas e preparadas para aplicação de outras tecnicas.

## Importação dos dados e recursos necessários

As bases de dados utilizadas para o desenvolvimento deste trabalho (microdados) podem ser encontradas no portal do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (INEP), o órgão responsável pelo ENEM, através do link: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem.

Cada microdado, por seu turno, contêm uma variedade de informações sobre os participantes coletadas ao longo de todo o processo do exame. Ao realizar o download, os usuários encontrarão, não apenas a própria base de dados, como também: as provas, gabaritos, informações sobre questões, notas, questionários respondidos pelos inscritos, documentos técnicos e, acima de tudo, um extenso dicionário relacionado ao conjunto de dados. Este dicionário se mostra especialmente relevante para o desenvolvimento deste trabalho uma vez que ele caracteriza objetivamente todas as colunas presentes na base de dados.

Vale ressaltar que os microdados estão formatados em arquivos de extesão ".csv" e, para os anos selecionados, cada tabela contêm cerca de 76 colunas. Devido à grande quantidade de colunas e à diversidade de possíveis respostas que cada coluna pode conter, torna-se inviável realizar uma caracterização completa da base neste trabalho. No entanto, todas as colunas relevantes para o desenvolvimento desta pesquisa terão seus significados explorados em momentos oportunos. Recomenda-se a leitura dos dicionários de dados referêntes aos anos propostos.

In [1]:
import pandas as pd

In [2]:
# Variáveis de controle

# ano = '2019'
# ano = '2020'
ano = '2021'
# ano = '2022'
# ano = '2023'

# Endereço dados brutos:
DATASET_ENEM_PATH = 'D:\Bases\MICRODADOS_ENEM_' + ano + '.csv'

# O novo arquivo será salvo neste endereço:
PATH_DATASET_TRATADO = 'D:\\BASES_PRE_PROCESSADAS\\PRE_PROCESSADOS_ENEM_' + ano + '.csv'

In [3]:
microdadosEnem = pd.read_csv(DATASET_ENEM_PATH, sep=';', encoding='ISO-8859-1')

In [4]:
microdadosEnem['TP_COR_RACA'].unique()

array([1, 3, 2, 4, 5, 0, 6], dtype=int64)

In [5]:
microdadosEnem.loc[microdadosEnem['TP_COR_RACA'] == 6]

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
456293,210051983991,2021,12,M,1,6,1,1,0,1,...,A,A,A,B,A,A,B,A,A,A
1367374,210051118185,2021,14,F,3,6,1,1,0,1,...,A,A,A,B,A,A,D,A,A,B
1645311,210051135416,2021,12,M,2,6,1,1,15,1,...,B,A,B,B,A,A,C,A,B,B
1873845,210054081568,2021,14,M,2,6,1,1,0,1,...,B,A,A,B,A,A,D,A,B,B
1903839,210053842982,2021,12,F,1,6,1,1,13,1,...,B,A,A,B,A,A,D,A,A,B


## Pré-processamento dos dados

O objetivo principal deste pré-processamento dos dados é preparar e otimizar os dados brutos para análise ou modelagem, visando melhorar a qualidade, a eficácia e a eficiência das etapas subsequentes.

### Eliminando características individuais, de baixa variância ou inexpressivas.

Por meio do dicionário de dados, sabe-se que as colunas abaixo possuem pouca relevância para o objetivo da análise. Isto porque algumas delas possuem variância igual a zero (como é o caso da coluna "NU_ANO" que estará sempre preenchida com o ano de realização da prova) e outras representam caracteristicas altamente individuais dos candidatos e, portanto, não são aptas para avaliação de nenhum tipo de tendência (como é o caso das colunas com prefixo "TX_" que são os vetores com as respostas objetivas para as diferentes áreas de conhecimento da prova de cada candidato). Há ainda um conjunto de colunas inexpressivas que tratam sobre a cor de prova do participante. Todas estas mencionadas serão removidas.

* "NU_INSCRICAO" - Número de inscrição (Individual),
* "NU_ANO" - Ano do Enem (Variância Zero),
* "TX_RESPOSTAS_CN" - Vetor com as respostas da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_RESPOSTAS_CH" - Vetor com as respostas da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_RESPOSTAS_LC" - Vetor com as respostas da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_RESPOSTAS_MT" - Vetor com as respostas da parte objetiva da prova de Matemática (Individual),
* "TX_GABARITO_CN" - Vetor com o gabarito da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_GABARITO_CH" - Vetor com o gabarito da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_GABARITO_LC" - Vetor com o gabarito da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_GABARITO_MT" - Vetor com o gabarito da parte objetiva da prova de Matemática (Individual),
* "CO_PROVA_CN" - Código do tipo de prova de Ciências da Natureza (Inexpressivo),
* "CO_PROVA_CH" - Código do tipo de prova de Ciências Humanas (Inexpressivo),
* "CO_PROVA_LC" - Código do tipo de prova de Linguagens e Códigos (Inexpressivo),
* "CO_PROVA_MT" - Código do tipo de prova de Matemática (Inexpressivo)
* "IN_TREINEIRO" - Indica se o inscrito fez a prova com intuito de apenas treinar seus conhecimentos (Inexpressivo)
* "TP_LINGUA" - Língua Estrangeira escolhida para realização da prova (Inexpressivo) * 

In [6]:
microdadosEnem.drop(columns=[
    'NU_ANO',
    'NU_INSCRICAO',
    'TX_RESPOSTAS_CN',
    'TX_RESPOSTAS_CH',
    'TX_RESPOSTAS_LC',
    'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN',
    'TX_GABARITO_CH',
    'TX_GABARITO_LC',
    'TX_GABARITO_MT',
    'CO_PROVA_CN',
    'CO_PROVA_CH',
    'CO_PROVA_LC',
    'CO_PROVA_MT',
    'IN_TREINEIRO',
    'TP_LINGUA'
], inplace=True)

microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,5,F,1,1,1,1,3,1,NaN,NaN,...,A,A,A,B,A,A,B,A,B,B
1,12,M,1,1,1,1,11,1,NaN,NaN,...,A,A,A,B,A,A,C,A,A,A
2,13,F,3,1,1,1,15,1,NaN,NaN,...,B,A,A,B,A,A,C,B,B,B
3,3,M,1,3,1,2,0,2,1.0,2304202.0,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,0,2,1.0,2311603.0,...,B,A,A,B,A,B,E,A,B,B


### Eliminando redundâncias

Além disso, irei eliminar colunas redundantes.

Exemplo: "NO_MUNICIPIO_RESIDENCIA" (nome do município) equivalente a "CO_MUNICIPIO_RESIDENCIA" (código do município).

In [7]:
microdadosEnem.drop(columns=[
    'NO_MUNICIPIO_ESC','CO_UF_ESC','SG_UF_ESC', # Todas estas colunas são iguais à CO_MUNICIPIO_ESC
    'NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA', # Todas estas colunas são iguais à CO_MUNICIPIO_PROVA
    'TP_ANO_CONCLUIU' # Ano de Conclusão do Ensino Médio - Essa informação pode ser mais facilmente extraída da coluna TP_ST_CONCLUSAO
], inplace=True)

microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,TP_DEPENDENCIA_ADM_ESC,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,5,F,1,1,1,1,1,NaN,NaN,NaN,...,A,A,A,B,A,A,B,A,B,B
1,12,M,1,1,1,1,1,NaN,NaN,NaN,...,A,A,A,B,A,A,C,A,A,A
2,13,F,3,1,1,1,1,NaN,NaN,NaN,...,B,A,A,B,A,A,C,B,B,B
3,3,M,1,3,1,2,2,1.0,2304202.0,2.0,...,A,A,A,B,A,A,B,A,B,B
4,2,F,1,3,1,2,2,1.0,2311603.0,2.0,...,B,A,A,B,A,B,E,A,B,B


### Tratamento de dados faltantes

In [8]:
microdadosEnem.isnull().values.any() # Verificando se existem valores nulos

True

* True == A base contém dados nulos;
* False == A base não contém dados nulos 

Para lidar com os dados ausentes no _dataset_, uma possível abordagem seria remover todas as linhas que contenham quaisquer valores nulos. Ao final deste processo, haveria um conjunto de dados compostos apenas por linhas que possuem todas as colunas preenchidas.

Apesar da simplicidade de implementação desta técnica, esta abordagem se mostrou inviável devido a alta perda de informações. Ao realizar um estudo de caso nas bases propostas, pude perceber que a implementação desta técnica ocarionaria na perda média de 70% dos registros presentes nas bases de dados.

Desse modo, será necessário adotar estratégias específicas para tratar cada coluna que contém valores nulos.
Primeiro é necessário compreender onde estão os valores nulos:

In [9]:
microdadosEnem.isnull().sum() # Verificando onde estão os valores nulos

TP_FAIXA_ETARIA                 0
TP_SEXO                         0
TP_ESTADO_CIVIL                 0
TP_COR_RACA                     0
TP_NACIONALIDADE                0
TP_ST_CONCLUSAO                 0
TP_ESCOLA                       0
TP_ENSINO                 2293004
CO_MUNICIPIO_ESC          2576026
TP_DEPENDENCIA_ADM_ESC    2576026
TP_LOCALIZACAO_ESC        2576026
TP_SIT_FUNC_ESC           2576026
CO_MUNICIPIO_PROVA              0
TP_PRESENCA_CN                  0
TP_PRESENCA_CH                  0
TP_PRESENCA_LC                  0
TP_PRESENCA_MT                  0
NU_NOTA_CN                1143988
NU_NOTA_CH                1011453
NU_NOTA_LC                1011453
NU_NOTA_MT                1143988
TP_STATUS_REDACAO         1011453
NU_NOTA_COMP1             1011453
NU_NOTA_COMP2             1011453
NU_NOTA_COMP3             1011453
NU_NOTA_COMP4             1011453
NU_NOTA_COMP5             1011453
NU_NOTA_REDACAO           1011453
Q001                            2
Q002          


##### ESTRATÉGIAS:

1. É possivel observar que há uma quantidade significativa de registros com informações ausentes relacionadas à **escola do candidato**. São elas:

    * "CO_MUNICIPIO_ESC" (Código do município da escola);
    * "TP_DEPENDENCIA_ADM_ESC" (Dependência administrativa da escola);
    * "TP_LOCALIZACAO_ESC" (Tipo de localização da escola); 
    * "TP_SIT_FUNC_ESC" (Situação de funcionamento da escola);
    * "TP_ENSINO" (Tipo de instituição de ensino);

Acredita-se que estas informações relacionadas a escola do candidato não eram de preenchimento obrigatório no momento cadastro do participante no exame. Isto justificaria a grande ausência de informações.

Dado que aproximadamente 70% das linhas da tabela total não possuem esses valores preenchidos, vou optar por remover essas colunas do modelo, pois não será possivel utiliza-las como objeto de ánalise confiável e nem fazer nenhum outro tipo de inferência preditiva ou classificativa.

Sendo assim, nenhum tipo de inferência ou analise sobre a escola do candidato será feita, mas conseguiremos preservar 70% a mais de dados.

_Uma sugestão de trabalho para o futuro é fazer analises relacionadas justamente as escolas dos participantes._

In [10]:
# Removendo as colunas não confiáveis

microdadosEnem.drop(columns=[
    'CO_MUNICIPIO_ESC',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC',
    'TP_ENSINO'
], inplace=True)

2. As colunas relacionadas as notas dos participantes também demonstram ter uma quantidade grande de dados nulos. 

Acredita-se que estas colunas possuam registros vazios devido a ausência do candidato no dia de aplicação do exame. Desta forma, **vou optar por prosseguir apenas com os participantes presentes nos dois dias de aplicação da prova**.

Ao optar por isto, é fato que estarei reduzindo minha quantidade de registros, mas poderei também descartar as colunas relacionadas a presença dos candidatos e simplificar meu modelo.

(Estas colunas deverão ter variância zero. Ou seja, apenas candidatos presentes)

_Uma sugestão de trabalho para o futuro é fazer a analise justamente destas linhas que estou desconsiderando._

In [11]:
# Removendo participantes ausêntes por meio da análise das notas

microdadosEnem.dropna(subset=['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], inplace=True)

In [12]:
# Validando se realmente posso remover as colunas de presença por meio da análise de variância.

colunasDePresença = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
microdadosEnem[colunasDePresença].var()

TP_PRESENCA_CN    0.0
TP_PRESENCA_CH    0.0
TP_PRESENCA_LC    0.0
TP_PRESENCA_MT    0.0
dtype: float64

In [13]:
# Dado que a hipotese foi validada, posso remover as colunas de presença para simplificar meu modelo

microdadosEnem.drop(columns=colunasDePresença, inplace=True)

##### REAVALIAÇÃO DA CONSISTÊNCIA DOS DADOS:

Se tudo tiver ocorrido conforme o planejado, não devem ter mais dados nulos na tabela

In [14]:
microdadosEnem.isnull().sum()

TP_FAIXA_ETARIA       0
TP_SEXO               0
TP_ESTADO_CIVIL       0
TP_COR_RACA           0
TP_NACIONALIDADE      0
TP_ST_CONCLUSAO       0
TP_ESCOLA             0
CO_MUNICIPIO_PROVA    0
NU_NOTA_CN            0
NU_NOTA_CH            0
NU_NOTA_LC            0
NU_NOTA_MT            0
TP_STATUS_REDACAO     0
NU_NOTA_COMP1         0
NU_NOTA_COMP2         0
NU_NOTA_COMP3         0
NU_NOTA_COMP4         0
NU_NOTA_COMP5         0
NU_NOTA_REDACAO       0
Q001                  1
Q002                  1
Q003                  1
Q004                  1
Q005                  1
Q006                  1
Q007                  1
Q008                  1
Q009                  1
Q010                  1
Q011                  1
Q012                  1
Q013                  1
Q014                  1
Q015                  1
Q016                  1
Q017                  1
Q018                  1
Q019                  1
Q020                  1
Q021                  1
Q022                  1
Q023            

Por fim, serão descartados os candidatos que possuirem alguma informação não preenchida no questionário socioeconômico principal.

**OBS**: Este procedimento está excluindo poucos dados do _dataset_. Isso porque não é comum que o questionário socioeconômico principal possua dados ausentes.

In [15]:
microdadosEnem.dropna(subset=[
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
], inplace=True)

In [16]:
microdadosEnem.shape

(2238106, 44)

## Transformação dos dados

Esta pesquisa visa realizar uma análise histórica das bases de dados do ENEM e identificar tendências temporais nas características de candidatos quando dividios em _clusters_.

Em outras palavras, queremos entender se as caracteristicas relacionadas a _clusters_ de candidatos com predominância de notas baixas e altas se alterou ao longo do tempo.

Dessa forma, é necessário adotar uma forma simples para mensurar se a nota do partipante foi alta ou baixa. Isso será de grande importância para realização das analises quando os candidados estiverem clusterizados.

A estratégia adotada será a de criação de uma nova coluna que contenha a média simples de todas as notas que o candidato obteve.

In [17]:
microdadosEnem['MEDIA_NOTAS'] = (
    microdadosEnem.NU_NOTA_CN +
    microdadosEnem.NU_NOTA_CH +
    microdadosEnem.NU_NOTA_LC +
    microdadosEnem.NU_NOTA_MT +
    microdadosEnem.NU_NOTA_REDACAO
)/5

In [18]:
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,CO_MUNICIPIO_PROVA,NU_NOTA_CN,NU_NOTA_CH,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,MEDIA_NOTAS
1,12,M,1,1,1,1,1,2704302,505.9,551.8,...,A,A,B,A,A,C,A,A,A,515.50
3,3,M,1,3,1,2,2,2304202,580.7,678.9,...,A,A,B,A,A,B,A,B,B,667.60
4,2,F,1,3,1,2,2,2300150,497.7,532.4,...,A,A,B,A,B,E,A,B,B,570.06
8,8,F,1,3,1,2,2,3106200,487.4,476.5,...,A,A,B,A,A,B,A,A,B,485.60
9,4,F,1,3,1,2,2,2927200,507.6,539.2,...,A,B,C,B,A,C,A,A,B,466.94


Agora, seguirei com o descarte das colunas referênte as notas dos candidatos nas áreas do conhecimento, dado ques estas colunas já estão representadas na coluna de média.

Além disso, removerei as colunas relacionadas as competências e ao status da redação, dado que estas colunas nada mais são que componentes já representados pela coluna de média.

Caso deixassemos estas colunas presentes no dataset, elas sempre seriam evidenciadas como os fatores mais relevantes para progressão das notas (obviamente).

_EX: Não possuir a redação com status de anulada sempre significará maior média de nota._ 

In [19]:
microdadosEnem.drop(columns=[
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP3',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5',
    'TP_STATUS_REDACAO'
], inplace=True)

In [20]:
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,CO_MUNICIPIO_PROVA,Q001,Q002,...,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,MEDIA_NOTAS
1,12,M,1,1,1,1,1,2704302,B,B,...,A,A,B,A,A,C,A,A,A,515.50
3,3,M,1,3,1,2,2,2304202,B,B,...,A,A,B,A,A,B,A,B,B,667.60
4,2,F,1,3,1,2,2,2300150,D,E,...,A,A,B,A,B,E,A,B,B,570.06
8,8,F,1,3,1,2,2,3106200,B,B,...,A,A,B,A,A,B,A,A,B,485.60
9,4,F,1,3,1,2,2,2927200,D,D,...,A,B,C,B,A,C,A,A,B,466.94


### Simplificando colunas com muitas possiveis opções

In [21]:
microdadosEnem['Q005'].replace(6, 5, inplace=True)
microdadosEnem['Q005'].replace(7, 5, inplace=True)
microdadosEnem['Q005'].replace(8, 5, inplace=True)
microdadosEnem['Q005'].replace(9, 5, inplace=True)
microdadosEnem['Q005'].replace(10, 5, inplace=True)
microdadosEnem['Q005'].replace(11, 5, inplace=True)
microdadosEnem['Q005'].replace(12, 5, inplace=True)
microdadosEnem['Q005'].replace(13, 5, inplace=True)
microdadosEnem['Q005'].replace(14, 5, inplace=True)
microdadosEnem['Q005'].replace(15, 5, inplace=True)
microdadosEnem['Q005'].replace(16, 5, inplace=True)
microdadosEnem['Q005'].replace(17, 5, inplace=True)
microdadosEnem['Q005'].replace(18, 5, inplace=True)
microdadosEnem['Q005'].replace(19, 5, inplace=True)
microdadosEnem['Q005'].replace(20, 5, inplace=True)

microdadosEnem['Q006'].replace('M', 'L', inplace=True)
microdadosEnem['Q006'].replace('N', 'L', inplace=True)
microdadosEnem['Q006'].replace('O', 'L', inplace=True)
microdadosEnem['Q006'].replace('P', 'L', inplace=True)
microdadosEnem['Q006'].replace('Q', 'L', inplace=True)

microdadosEnem['TP_FAIXA_ETARIA'].replace(14, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(15, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(16, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(17, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(18, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(19, 13, inplace=True)
microdadosEnem['TP_FAIXA_ETARIA'].replace(20, 13, inplace=True)

In [22]:
def mapear_macro_regiao(codigo):
    primeiro_digito = str(codigo)[0]
    if primeiro_digito == '1':
        return 'NORTE'
    elif primeiro_digito == '2':
        return 'NORDESTE'
    elif primeiro_digito == '3':
        return 'SUDESTE'
    elif primeiro_digito == '4':
        return 'SUL'
    elif primeiro_digito == '5':
        return 'CENTRO_OESTE'
    else:
        return 'UNKNOWN'  # Caso algum código não esteja nos intervalos esperados

# Aplicando a função à coluna 'CO_MUNICIPIO_PROVA'
microdadosEnem['MACRO_REGIAO'] = microdadosEnem['CO_MUNICIPIO_PROVA'].apply(mapear_macro_regiao)
microdadosEnem.drop(columns=['CO_MUNICIPIO_PROVA'], inplace=True)
microdadosEnem.head()

,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,Q001,Q002,Q003,...,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,MEDIA_NOTAS,MACRO_REGIAO
1,12,M,1,1,1,1,1,B,B,B,...,A,B,A,A,C,A,A,A,515.50,NORDESTE
3,3,M,1,3,1,2,2,B,B,B,...,A,B,A,A,B,A,B,B,667.60,NORDESTE
4,2,F,1,3,1,2,2,D,E,F,...,A,B,A,B,E,A,B,B,570.06,NORDESTE
8,8,F,1,3,1,2,2,B,B,B,...,A,B,A,A,B,A,A,B,485.60,SUDESTE
9,4,F,1,3,1,2,2,D,D,D,...,B,C,B,A,C,A,A,B,466.94,NORDESTE


### Tratando variáveis categóricas

O tratamento adequado de variáveis categóricas desempenha um papel crucial na análise e modelagem de dados. As variáveis categóricas representam características qualitativas que não podem ser diretamente utilizadas em algoritmos de aprendizado de máquina.

#### _get_dummies_ VS _OneHotEncoder_

O OneHotEncoder e o get_dummies são técnicas comuns para lidar com variáveis categóricas em análises de dados e modelagem estatística. Ambos convertem variáveis categóricas em formato numérico para que possam ser utilizadas em algoritmos de aprendizado de máquina. A principal semelhança entre eles é que ambos criam variáveis indicadoras para cada categoria da variável categórica, transformando-as em variáveis binárias (0s e 1s).

Por exemplo: se existir uma coluna chamada "Cor" com categorias "Vermelho", "Verde" e "Azul". Ambas as funções poderiam criar três novas colunas: "Cor_Vermelho", "Cor_Verde" e "Cor_Azul". Cada linha terá um valor 1 na coluna correspondente à cor daquela linha e 0 nas outras colunas. Isso permite que as informações categóricas sejam tratadas como variáveis numéricas, facilitando a análise estatística e o uso em modelos de machine learning (ML).

Ainda que os métodos possuam grandes semelhanças, seus casos de uso são um pouco distintos. O OneHotEncoder faz parte da biblioteca Scikit-Learn e é frequentemente aplicado como parte de um pipeline de pré-processamento de dados. Este método é especialmente utilizado quando o desenvolvimento está relacionado ao aprendizado de máquina. Isso porque o OneHotEncoder tem uma gestão melhor de variáveis não mapeadas - evitando erros - e o processamento que foi criado pelo OneHotEncoder pode ser armazenado em arquivos pickle, permitindo o uso em outros projetos.

Por outro lado, o get_dummies, que faz parte da biblioteca pandas em Python, é mais simples e direto. Sua única responsabilidade é criar colunas separadas para cada categoria da variável categórica. Descarta-se a complexidade adicional de integração com pipelines, execução de etapas de treinamento, etc.

Certamente o algoritmo OneHotEncoder possui mais features e possibilidades de utilização, no entanto, para o trabalho atual, seguiremos com a simplicidade do get_dummies dado que: não há o foco no desenvolvimento do modelo de ML, as variáveis categóricas constituem um escopo asseguradamente fechado, os pontos positivos do OneHotEncoder não serão aproveitados e o custo computacional dele se demonstrou maior.

#### _get_dummies_

In [23]:
# Lista de colunas para incluir no get_dummies
colunas_para_dummies = [col for col in microdadosEnem.columns if col not in ['MEDIA_NOTAS']]

# Aplicando o get_dummies nas colunas desejadas
microdadosEnem = pd.get_dummies(microdadosEnem, columns=colunas_para_dummies)

In [41]:
print(len(colunas_para_dummies))

33


In [24]:
microdadosEnem.head()

,MEDIA_NOTAS,TP_FAIXA_ETARIA_1,TP_FAIXA_ETARIA_2,TP_FAIXA_ETARIA_3,TP_FAIXA_ETARIA_4,TP_FAIXA_ETARIA_5,TP_FAIXA_ETARIA_6,TP_FAIXA_ETARIA_7,TP_FAIXA_ETARIA_8,TP_FAIXA_ETARIA_9,...,Q024_C,Q024_D,Q024_E,Q025_A,Q025_B,MACRO_REGIAO_CENTRO_OESTE,MACRO_REGIAO_NORDESTE,MACRO_REGIAO_NORTE,MACRO_REGIAO_SUDESTE,MACRO_REGIAO_SUL
1,515.50,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,False
3,667.60,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
4,570.06,False,True,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False
8,485.60,False,False,False,False,False,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
9,466.94,False,False,False,True,False,False,False,False,False,...,False,False,False,False,True,False,True,False,False,False


In [25]:
microdadosEnem.shape

(2238106, 169)

In [26]:
print(microdadosEnem.columns.values)

['MEDIA_NOTAS' 'TP_FAIXA_ETARIA_1' 'TP_FAIXA_ETARIA_2' 'TP_FAIXA_ETARIA_3'
 'TP_FAIXA_ETARIA_4' 'TP_FAIXA_ETARIA_5' 'TP_FAIXA_ETARIA_6'
 'TP_FAIXA_ETARIA_7' 'TP_FAIXA_ETARIA_8' 'TP_FAIXA_ETARIA_9'
 'TP_FAIXA_ETARIA_10' 'TP_FAIXA_ETARIA_11' 'TP_FAIXA_ETARIA_12'
 'TP_FAIXA_ETARIA_13' 'TP_SEXO_F' 'TP_SEXO_M' 'TP_ESTADO_CIVIL_0'
 'TP_ESTADO_CIVIL_1' 'TP_ESTADO_CIVIL_2' 'TP_ESTADO_CIVIL_3'
 'TP_ESTADO_CIVIL_4' 'TP_COR_RACA_0' 'TP_COR_RACA_1' 'TP_COR_RACA_2'
 'TP_COR_RACA_3' 'TP_COR_RACA_4' 'TP_COR_RACA_5' 'TP_COR_RACA_6'
 'TP_NACIONALIDADE_0' 'TP_NACIONALIDADE_1' 'TP_NACIONALIDADE_2'
 'TP_NACIONALIDADE_3' 'TP_NACIONALIDADE_4' 'TP_ST_CONCLUSAO_1'
 'TP_ST_CONCLUSAO_2' 'TP_ST_CONCLUSAO_3' 'TP_ST_CONCLUSAO_4' 'TP_ESCOLA_1'
 'TP_ESCOLA_2' 'TP_ESCOLA_3' 'Q001_A' 'Q001_B' 'Q001_C' 'Q001_D' 'Q001_E'
 'Q001_F' 'Q001_G' 'Q001_H' 'Q002_A' 'Q002_B' 'Q002_C' 'Q002_D' 'Q002_E'
 'Q002_F' 'Q002_G' 'Q002_H' 'Q003_A' 'Q003_B' 'Q003_C' 'Q003_D' 'Q003_E'
 'Q003_F' 'Q004_A' 'Q004_B' 'Q004_C' 'Q004_D' 'Q004_E' '

In [27]:
microdadosEnem.to_csv(PATH_DATASET_TRATADO, index=False)